#sUBIR DATASET

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()  # Selecciona el archivo kaggle.json

{}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d alvarobaru/dog-emotion-dataset
!unzip -q dog-emotion-dataset.zip -d dog_emotion

403 Client Error: Forbidden for url: https://www.kaggle.com/api/v1/datasets/metadata/alvarobaru/dog-emotion-dataset
unzip:  cannot find or open dog-emotion-dataset.zip, dog-emotion-dataset.zip.zip or dog-emotion-dataset.zip.ZIP.


In [ ]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split

# Ruta al CSV con las etiquetas
csv_path = '/ruta/labels.csv'

# Carpeta donde están las imágenes organizadas por clases
source_dir = '/ruta/dataset_original'

# Carpeta de salida con estructura train/val por clase
output_dir = '/ruta/dog_emotion_dataset'
train_dir = os.path.join(output_dir, 'train')
val_dir = os.path.join(output_dir, 'val')

# Crear directorios si no existen
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Leer el CSV
df = pd.read_csv(csv_path)

# Dividir en train (80%) y val (20%) con


#Implementación

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader


In [ ]:
dataset_path = '/content/drive/MyDrive/dog_emotion_dataset.zip'
!unzip -q "/content/drive/MyDrive/dog_emotion_dataset.zip" -d /content/dog_emotion_dataset
import os
print(os.listdir('/content/dog_emotion_dataset'))

['Dog Emotion']


In [ ]:
!ls /content/dog_emotion_dataset/DogEmotion

angry  happy  labels.csv  relaxed  sad


In [ ]:

# Transformaciones (ajusta según tu modelo)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Tamaño esperado por muchos modelos CNN
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImagenNet stats
])

# Cargar dataset
dataset = ImageFolder(root='/content/dog_emotion_dataset/DogEmotion', transform=transform)

# Crear DataLoader
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
images, labels = next(iter(dataloader))
print(f"Tamaño del batch: {images.shape}")  # Debería ser [32, 3, 224, 224]
print(f"Etiquetas: {labels}")

Tamaño del batch: torch.Size([32, 3, 224, 224])
Etiquetas: tensor([2, 0, 0, 0, 3, 1, 1, 3, 0, 2, 2, 1, 2, 0, 3, 3, 2, 2, 0, 3, 2, 0, 2, 3,
        0, 3, 0, 0, 3, 1, 2, 0])


##Entrenar modelo simplificado

In [ ]:
# Crear carpeta para checkpoints si no existe
checkpoint_dir = 'checkpoints_resnet18'
os.makedirs(checkpoint_dir, exist_ok=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}") # Optional: Print the device being used
# Modelo
model = torchvision.models.resnet18(pretrained=True)
model.fc = nn.Linear(512, len(dataset.classes))  # Ajustar la última capa
model = model.to(device)

# Pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Bucle de entrenamiento
for epoch in range(5):
    model.train()
    running_loss = 0.0

    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(dataloader.dataset)
    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}")

    # Guardar modelo por época
    checkpoint_path = os.path.join(checkpoint_dir, f'resnet18_epoch_{epoch+1}.pth')
    torch.save({
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': epoch_loss
    }, checkpoint_path)


Using device: cpu


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 0.8576
Epoch 2, Loss: 0.6282
Epoch 3, Loss: 0.4782
Epoch 4, Loss: 0.3856
Epoch 5, Loss: 0.3145


In [ ]:
import torch

# Guardar el modelo (estructura + pesos)
torch.save(model.state_dict(), 'dog_emotion_model.pth')

# Opcional: Guardar todo el modelo (incluyendo arquitectura)
torch.save(model, 'dog_emotion_model_full.pth')

In [ ]:
from google.colab import files
files.download('dog_emotion_model.pth')  # Reemplaza con tu nombre de archivo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Evaluar o testear

In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Mismas transformaciones que en entrenamiento
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_dataset = ImageFolder(root='/content/dog_emotion_dataset/DogEmotion', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
model.eval()  # Modo evaluación (desactiva dropout, batch norm, etc.)
correct = 0
total = 0

with torch.no_grad():  # Desactiva el cálculo de gradientes
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)  # Clase predicha
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Precisión en el conjunto de prueba: {accuracy:.2f}%')

Precisión en el conjunto de prueba: 89.70%
